In [1]:
import os

import numpy as np

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [20]:
TEST = True

table = 'movement_model_inference_m2_a2_v3'

table = table if not TEST else f'{table}_test'
data = read_data_w_cache(
    f'select * from {table}'
)
data = data[data['_selected'].isin([True, False])]
data['log_prob'] = np.log(data['probability'])
data.head()

,_decision,_choice,_selected,mixed_layer_thickness,net_primary_production,water_heading,movement_heading,distance,origin_h3_index,next_h3_index,...,normed_log_npp,tag_key,log_odds,odds,probability,experiment_name,run_id,_train,_partition,log_prob
0,32,2,False,17.446415,33.689568,-2.445673,2.451028,79.468605,840c53bffffffff,840c53bffffffff,...,1.531594,159020,-7.049622,0.000868,0.000134,movement-model-m2-a2-v3,7d45db15389716d7da044cea141eb9276df828c54291d4...,False,0,-8.917493
1,14,2,True,26.568059,20.960207,2.022152,0.000000,0.000000,840c503ffffffff,840c503ffffffff,...,1.057050,159020,1.483967,4.410406,0.780713,movement-model-m2-a2-v3,7d45db15389716d7da044cea141eb9276df828c54291d4...,False,0,-0.247547
2,16,5,False,10.130819,16.027370,-2.386148,1.509298,71.457489,840c503ffffffff,840c503ffffffff,...,0.788737,159020,-6.432607,0.001608,0.000327,movement-model-m2-a2-v3,7d45db15389716d7da044cea141eb9276df828c54291d4...,False,0,-8.025569
3,12,5,False,10.780996,18.490732,-2.303153,1.509298,71.457489,840c503ffffffff,840c503ffffffff,...,0.931700,159020,-6.460110,0.001565,0.000325,movement-model-m2-a2-v3,7d45db15389716d7da044cea141eb9276df828c54291d4...,False,0,-8.031950
4,14,17,False,17.160476,31.911448,-2.317874,-2.839730,77.210167,840c503ffffffff,840c503ffffffff,...,1.477372,159020,-5.998731,0.002482,0.000439,movement-model-m2-a2-v3,7d45db15389716d7da044cea141eb9276df828c54291d4...,False,0,-7.730245


In [21]:
final = (
    data[data['_selected']]
    .groupby(['_individual', '_train'])[['log_prob']]
    .mean().reset_index()
    .groupby('_train')[['log_prob']].mean()
)
final['GMP'] = np.exp(final['log_prob'])
final

,log_prob,GMP
_train,,
False,-1.064232,0.344993
